In [ ]:
import numpy as np
import plotly.graph_objects as go
# import soundfile as sf
import wave
import sys
import sound_card_tools as sct

# Open the wav file
file_path = '/home/user/data/Exercise_3d_PAM_Rx_RRC_sc.wav'
                    

# Open the wav file
wav_file = wave.open(file_path, 'r')

# Extract Raw Audio from Wav File
signal = wav_file.readframes(-1)
signal = np.frombuffer(signal, dtype=np.int16)

# Get the number of channels
channels = wav_file.getnchannels()

# Split the data into channels
if channels == 3:
    signal = np.reshape(signal, (-1, channels))
    sc_output = signal[:, 0]
    sc_input = signal[:, 1]
    sc_filter_input = signal[:, 2]
else:
    print('The file does not have 3 channels it has {channels} channels')
    sys.exit(0)
sample_rate = wav_file.getframerate()

[delay_idx, delay_seconds]= sct.find_delay_by_amplitude(sc_output, sc_input, sample_rate,noise_window=1000, threshold_factor=15)
[delay_filter_idx, delay_filter_seconds]= sct.find_delay_by_amplitude(sc_output, sc_filter_input, sample_rate,noise_window=1000, threshold_factor=15)
print(f'The delay is {delay_seconds} seconds or {delay_idx} samples. The delay of the filter is {delay_filter_seconds} seconds or {delay_filter_idx} samples')


# Create interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(y=sc_output, mode='lines', name='sc_output'))
fig.add_trace(go.Scatter(y=sc_input, mode='lines', name='sc_input'))
fig.add_trace(go.Scatter(y=sc_filter_input, mode='lines', name='sc_filter_input'))
# add location of the delay
fig.add_trace(go.Scatter(x=[delay_idx, delay_idx], y=[np.min(signal), np.max(signal)], mode='lines', name='Delay'))
fig.add_trace(go.Scatter(x=[delay_filter_idx, delay_filter_idx], y=[np.min(signal), np.max(signal)], mode='lines', name='Delay Filter'))
fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
fig.show()



In [ ]:
# cut both sc_input and sc_output to the same length with sct.cut_signals
delay_cut_idx = delay_idx  - 12 
delay_filter_cut_idx = delay_filter_idx  - 32 - 20 - 3
sc_cut_input, sc_cut_output = sct.cut_signals(sc_input, sc_output, delay_cut_idx)
sc_cut_filter_input, sc_cut_filter_output = sct.cut_signals(sc_filter_input, sc_output, delay_filter_cut_idx)

# plot the cut signals
fig = go.Figure()
fig.add_trace(go.Scatter(y=sc_cut_output, mode='lines', name='sc_output'))
fig.add_trace(go.Scatter(y=sc_cut_input, mode='lines', name='sc_input'))
fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
fig.show()

# plot the cut signals
fig = go.Figure()
fig.add_trace(go.Scatter(y=sc_cut_filter_output, mode='lines', name='sc_cut_filter_output'))
fig.add_trace(go.Scatter(y=sc_cut_filter_input, mode='lines', name='sc_cut_filter_input'))
fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
fig.show()


In [ ]:
# # Find the delay using correlation
# correlation = np.correlate(sc_output, sc_input, mode='full')
# delay = np.argmax(correlation) - (len(sc_output) - 1)
# print(f'Delay between channels: {delay} samples')

# # plot the correlation
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=correlation, mode='lines', name='Correlation'))
# fig.update_layout(title='Correlation Plot', xaxis_title='Sample Index', yaxis_title='Correlation')
# fig.show()



# # Create interactive plot
# fig = go.Figure()
# if delay < 0:
#     fig.add_trace(go.Scatter(y=sc_output, mode='lines', name='sc_output'))
#     fig.add_trace(go.Scatter(y=sc_input[(-delay):], mode='lines', name='sc_input'))
# else:
#     fig.add_trace(go.Scatter(y=sc_output[delay:], mode='lines', name='sc_output'))
#     fig.add_trace(go.Scatter(y=sc_input, mode='lines', name='sc_input'))
# fig.update_layout(title='Aligned Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
# fig.show()

# # cut_signals(sc_output, sc_input, 
# #                 sc_output_start_index, sc_input_start_index):